In [2]:
import os
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from datetime import datetime
from pandas.api.types import CategoricalDtype
import sqlite3
pd.set_option('display.max_columns', 500)


In [17]:
#get data from csv
prepared_matches = pd.read_csv("prepared_matches.csv", low_memory = False)

## Data preprocessing part 2

edit the collected data

In [18]:
#show null values
pd.options.display.max_rows = 30#400
prepared_matches.isnull().sum()
#15745 rows

Unnamed: 0                          0
id_int                              0
id_int_home                         0
id_int_away                         0
competition                         0
match_day                        5463
date                                0
goals_home                          0
goals_away                          0
annotation                      15468
country_id_int_home                 0
country_id_string_home              0
country_name_home                   0
country_id_int_away                 3
country_id_string_away              3
                                ...  
hp_12_total_time_injured        15745
hp_12_since_last_injury         15745
ap_12_starting                  15744
ap_12_id_int_player             15744
ap_12_id_string                 15744
ap_12_name                      15744
ap_12_position                  15744
ap_12_age                       15744
ap_12_height                    15745
ap_12_debut                     15745
ap_12_countr

In [19]:
#add the information about the winning team to the table

pd.options.mode.chained_assignment = None  # default='warn'

prepared_matches["home_is_winner"] = 0
for index, row in prepared_matches.iterrows():
    if prepared_matches["goals_home"][index] > prepared_matches["goals_away"][index]:
        prepared_matches["home_is_winner"][index] = 1
    elif prepared_matches["goals_home"][index] < prepared_matches["goals_away"][index]:
         prepared_matches["home_is_winner"][index] = 0
    else:
        prepared_matches["home_is_winner"][index] = 2
    
prepared_matches.head(100)

,Unnamed: 0,id_int,id_int_home,id_int_away,competition,match_day,date,goals_home,goals_away,annotation,country_id_int_home,country_id_string_home,country_name_home,country_id_int_away,country_id_string_away,country_name_away,rank_home,country_full_home,country_abrv_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,confederation_home,rank_date_home,rank_away,country_full_away,country_abrv_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,confederation_away,rank_date_away,id_int_match,hp_1_starting,hp_1_id_int_player,hp_1_id_string,hp_1_name,hp_1_position,hp_1_age,hp_1_height,hp_1_debut,hp_1_country_id_int,hp_1_country_id_string_away,hp_1_injury,hp_1_total_time_injured,hp_1_since_last_injury,hp_2_starting,hp_2_id_int_player,hp_2_id_string,hp_2_name,hp_2_position,hp_2_age,hp_2_height,hp_2_debut,hp_2_country_id_int,hp_2_country_id_string_away,hp_2_injury,hp_2_total_time_injured,hp_2_since_last_injury,hp_3_starting,hp_3_id_int_player,hp_3_id_string,hp_3_name,hp_3_position,hp_3_age,hp_3_height,hp_3_debut,hp_3_country_id_int,hp_3_country_id_string_away,hp_3_injury,hp_3_total_time_injured,hp_3_since_last_injury,hp_4_starting,hp_4_id_int_player,hp_4_id_string,hp_4_name,hp_4_position,hp_4_age,hp_4_height,hp_4_debut,hp_4_country_id_int,hp_4_country_id_string_away,hp_4_injury,hp_4_total_time_injured,hp_4_since_last_injury,hp_5_starting,hp_5_id_int_player,hp_5_id_string,hp_5_name,hp_5_position,hp_5_age,hp_5_height,hp_5_debut,hp_5_country_id_int,hp_5_country_id_string_away,hp_5_injury,hp_5_total_time_injured,hp_5_since_last_injury,hp_6_starting,hp_6_id_int_player,hp_6_id_string,hp_6_name,hp_6_position,hp_6_age,hp_6_height,hp_6_debut,hp_6_country_id_int,hp_6_country_id_string_away,hp_6_injury,hp_6_total_time_injured,hp_6_since_last_injury,hp_7_starting,hp_7_id_int_player,hp_7_id_string,hp_7_name,hp_7_position,hp_7_age,hp_7_height,hp_7_debut,hp_7_country_id_int,hp_7_country_id_string_away,hp_7_injury,hp_7_total_time_injured,hp_7_since_last_injury,hp_8_starting,hp_8_id_int_player,hp_8_id_string,hp_8_name,hp_8_position,hp_8_age,hp_8_height,hp_8_debut,hp_8_country_id_int,hp_8_country_id_string_away,hp_8_injury,hp_8_total_time_injured,hp_8_since_last_injury,hp_9_starting,hp_9_id_int_player,hp_9_id_string,hp_9_name,hp_9_position,hp_9_age,hp_9_height,hp_9_debut,hp_9_country_id_int,hp_9_country_id_string_away,hp_9_injury,hp_9_total_time_injured,hp_9_since_last_injury,hp_10_starting,hp_10_id_int_player,hp_10_id_string,hp_10_name,hp_10_position,hp_10_age,hp_10_height,hp_10_debut,hp_10_country_id_int,hp_10_country_id_string_away,hp_10_injury,hp_10_total_time_injured,hp_10_since_last_injury,hp_11_starting,hp_11_id_int_player,hp_11_id_string,hp_11_name,hp_11_position,hp_11_age,hp_11_height,hp_11_debut,hp_11_country_id_int,hp_11_country_id_string_away,hp_11_injury,hp_11_total_time_injured,hp_11_since_last_injury,ap_1_starting,ap_1_id_int_player,ap_1_id_string,ap_1_name,ap_1_position,ap_1_age,ap_1_height,ap_1_debut,ap_1_country_id_int,ap_1_country_id_string_away,ap_1_injury,ap_1_total_time_injured,ap_1_since_last_injury,ap_2_starting,ap_2_id_int_player,ap_2_id_string,ap_2_name,ap_2_position,ap_2_age,ap_2_height,ap_2_debut,ap_2_country_id_int,ap_2_country_id_string_away,ap_2_injury,ap_2_total_time_injured,ap_2_since_last_injury,ap_3_starting,ap_3_id_int_player,ap_3_id_string,ap_3_name,ap_3_position,ap_3_age,ap_3_height,ap_3_debut,ap_3_country_id_int,ap_3_country_id_string_away,ap_3_injury,ap_3_total_time_injured,ap_3_since_last_injury,ap_4_starting,ap_4_id_int_player,ap_4_id_string,ap_4_name,ap_4_position,ap_4_age,ap_4_height,ap_4_debut,ap_4_country_id_in

In [20]:
prepared_matches_train = prepared_matches
len(prepared_matches_train)

15745

In [21]:
prepared_matches_train = prepared_matches_train.drop(["Unnamed: 0"],  axis=1)

In [22]:
#drop all columns with unneccessary information
prepared_matches_train = prepared_matches_train.drop(["country_id_string_home", "country_id_int_home", "country_id_string_away", "country_id_int_away", 
                             "hp_1_id_string", "hp_1_name", "hp_1_country_id_string_away", "hp_1_position",
                             "hp_2_id_string", "hp_2_name", "hp_2_country_id_string_away", "hp_2_position",
                             "hp_3_id_string", "hp_3_name", "hp_3_country_id_string_away", "hp_3_position",
                             "hp_4_id_string", "hp_4_name", "hp_4_country_id_string_away", "hp_4_position",
                             "hp_5_id_string", "hp_5_name", "hp_5_country_id_string_away", "hp_5_position",
                             "hp_6_id_string", "hp_6_name", "hp_6_country_id_string_away", "hp_6_position",
                             "hp_7_id_string", "hp_7_name", "hp_7_country_id_string_away", "hp_7_position",
                             "hp_8_id_string", "hp_8_name", "hp_8_country_id_string_away", "hp_8_position",
                             "hp_9_id_string", "hp_9_name", "hp_9_country_id_string_away", "hp_9_position",
                             "hp_10_id_string", "hp_10_name", "hp_10_country_id_string_away", "hp_10_position",
                             "hp_11_id_string", "hp_11_name", "hp_11_country_id_string_away", "hp_11_position",
                             "ap_1_id_string", "ap_1_name", "ap_1_country_id_string_away", "ap_1_position",
                             "ap_2_id_string", "ap_2_name", "ap_2_country_id_string_away", "ap_2_position",
                             "ap_3_id_string", "ap_3_name", "ap_3_country_id_string_away", "ap_3_position",
                             "ap_4_id_string", "ap_4_name", "ap_4_country_id_string_away", "ap_4_position",
                             "ap_5_id_string", "ap_5_name", "ap_5_country_id_string_away", "ap_5_position",
                             "ap_6_id_string", "ap_6_name", "ap_6_country_id_string_away", "ap_6_position",
                             "ap_7_id_string", "ap_7_name", "ap_7_country_id_string_away", "ap_7_position",
                             "ap_8_id_string", "ap_8_name", "ap_8_country_id_string_away", "ap_8_position",
                             "ap_9_id_string", "ap_9_name", "ap_9_country_id_string_away", "ap_9_position",
                             "ap_10_id_string", "ap_10_name", "ap_10_country_id_string_away", "ap_10_position",
                             "ap_11_id_string", "ap_11_name", "ap_11_country_id_string_away", "ap_11_position",],  axis=1)

prepared_matches_train = prepared_matches_train.drop(["hp_1_starting",'hp_1_age', 
                                                      "hp_2_starting",'hp_2_age', 
                                                      "hp_3_starting",'hp_3_age', 
                                                      "hp_4_starting",'hp_4_age', 
                                                      "hp_5_starting",'hp_5_age', 
                                                      "hp_6_starting",'hp_6_age', 
                                                      "hp_7_starting",'hp_7_age', 
                                                      "hp_8_starting",'hp_8_age', 
                                                      "hp_9_starting",'hp_9_age', 
                                                      "hp_10_starting",'hp_10_age',
                                                      "hp_11_starting",'hp_11_age',
                                                      "ap_1_starting",'ap_1_age',
                                                      "ap_2_starting",'ap_2_age', 
                                                      "ap_3_starting",'ap_3_age', 
                                                      "ap_4_starting",'ap_4_age', 
                                                      "ap_5_starting",'ap_5_age', 
                                                      "ap_6_starting",'ap_6_age', 
                                                      "ap_7_starting",'ap_7_age', 
                                                      "ap_8_starting",'ap_8_age', 
                                                      "ap_9_starting",'ap_9_age', 
                                                      "ap_10_starting",'ap_10_age',
                                                      "ap_11_starting",'ap_11_age'],  axis=1)

prepared_matches_train = prepared_matches_train.drop(["country_name_home","country_name_away", "country_full_home", "country_abrv_home",  "country_full_away", "country_abrv_away"],  axis=1)

prepared_matches_train.head(10)

,id_int,id_int_home,id_int_away,competition,match_day,date,goals_home,goals_away,annotation,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,confederation_home,rank_date_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,confederation_away,rank_date_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_debut,hp_1_country_id_int,hp_1_injury,hp_1_total_time_injured,hp_1_since_last_injury,hp_2_id_int_player,hp_2_height,hp_2_debut,hp_2_country_id_int,hp_2_injury,hp_2_total_time_injured,hp_2_since_last_injury,hp_3_id_int_player,hp_3_height,hp_3_debut,hp_3_country_id_int,hp_3_injury,hp_3_total_time_injured,hp_3_since_last_injury,hp_4_id_int_player,hp_4_height,hp_4_debut,hp_4_country_id_int,hp_4_injury,hp_4_total_time_injured,hp_4_since_last_injury,hp_5_id_int_player,hp_5_height,hp_5_debut,hp_5_country_id_int,hp_5_injury,hp_5_total_time_injured,hp_5_since_last_injury,hp_6_id_int_player,hp_6_height,hp_6_debut,hp_6_country_id_int,hp_6_injury,hp_6_total_time_injured,hp_6_since_last_injury,hp_7_id_int_player,hp_7_height,hp_7_debut,hp_7_country_id_int,hp_7_injury,hp_7_total_time_injured,hp_7_since_last_injury,hp_8_id_int_player,hp_8_height,hp_8_debut,hp_8_country_id_int,hp_8_injury,hp_8_total_time_injured,hp_8_since_last_injury,hp_9_id_int_player,hp_9_height,hp_9_debut,hp_9_country_id_int,hp_9_injury,hp_9_total_time_injured,hp_9_since_last_injury,hp_10_id_int_player,hp_10_height,hp_10_debut,hp_10_country_id_int,hp_10_injury,hp_10_total_time_injured,hp_10_since_last_injury,hp_11_id_int_player,hp_11_height,hp_11_debut,hp_11_country_id_int,hp_11_injury,hp_11_total_time_injured,hp_11_since_last_injury,ap_1_id_int_player,ap_1_height,ap_1_debut,ap_1_country_id_int,ap_1_injury,ap_1_total_time_injured,ap_1_since_last_injury,ap_2_id_int_player,ap_2_height,ap_2_debut,ap_2_country_id_int,ap_2_injury,ap_2_total_time_injured,ap_2_since_last_injury,ap_3_id_int_player,ap_3_height,ap_3_debut,ap_3_country_id_int,ap_3_injury,ap_3_total_time_injured,ap_3_since_last_injury,ap_4_id_int_player,ap_4_height,ap_4_debut,ap_4_country_id_int,ap_4_injury,ap_4_total_time_injured,ap_4_since_last_injury,ap_5_id_int_player,ap_5_height,ap_5_debut,ap_5_country_id_int,ap_5_injury,ap_5_total_time_injured,ap_5_since_last_injury,ap_6_id_int_player,ap_6_height,ap_6_debut,ap_6_country_id_int,ap_6_injury,ap_6_total_time_injured,ap_6_since_last_injury,ap_7_id_int_player,ap_7_height,ap_7_debut,ap_7_country_id_int,ap_7_injury,ap_7_total_time_injured,ap_7_since_last_injury,ap_8_id_int_player,ap_8_height,ap_8_debut,ap_8_country_id_int,ap_8_injury,ap_8_total_time_injured,ap_8_since_last_injury,ap_9_id_int_player,ap_9_height,ap_9_debut,ap_9_country_id_int,ap_9_injury,ap_9_total_time_injured,ap_9_since_last_injury,ap_10_id_int_player,ap_10_height,ap_10_debut,ap_10_country_id_int,ap_10_injury,ap_10_total_time_injured,ap_10_since_last_injury,ap_11_id_int_player,ap_11_height,ap_11_debut,ap_11_country_id_int,ap_11_injury,ap_11_total_time_injured,ap_11_since_last_injury,hp_12_starting,hp_12_id_int_player,hp_12_id_string,hp_12_name,hp_12_position,hp_12_age,hp_12_height,hp_12_debut,hp_12_country_id_int,hp_12_country_id_string_away,hp_12_injury,hp_12_total_time_injured,hp_12_since_last_injury,ap_12_starting,ap_12_id_int_player,ap_12_id_string,ap_12_name,ap_12_position,ap_12_age,ap_12_height,ap_12_debut,ap_12_country_id_int,ap_12_country_id_string_away,ap_12_injury,ap_12_total_time_injured,ap_12_since_last_injury,home_is_winner
0,1152440,3375,3379,Freundschaftsspiele,NaN,1980-01-23,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [23]:
#drop all columns with more than 10000 NaN (of 15000)
for column in prepared_matches_train:
    if prepared_matches_train[column].isnull().sum() > 10000:
        prepared_matches_train =  prepared_matches_train.drop(column, axis=1)

prepared_matches_train.head(10)

,id_int,id_int_home,id_int_away,competition,match_day,date,goals_home,goals_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,confederation_home,rank_date_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,confederation_away,rank_date_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner
0,1152440,3375,3379,Freundschaftsspiele,NaN,1980-01-23,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1152440.0,135084.0,1.80,3375.0,117603.0,1.80,3375.0,159968.0,1.77,3375.0,135089.0,1.85,3375.0,135099.0,1.83,3375.0,141272.0,1.79,3375.0,135088.0,1.78,3375.0,141276.0,1.71,3375.0,135096.0,1.69,3375.0,117599.0,1.83,3375.0,135092.0,1.75,3375.0,135084.0,1.80,3375.0,117603.0,1.80,3375.0,159968.0,1.77,3375.0,135089.0,1.85,3375.0,135099.0,1.83,3375.0,141272.0,1.79,3375.0,135088.0,1.78,3375.0,141276.0,1.71,3375.0,135096.0,1.69,3375.0,117599.0,1.83,3375.0,135092.0,1.75,3375.0,1
1,2380975,3299,3509,EM-Qualifikation,Gruppe 1,1980-02-06,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2380975.0,130301.0,1.83,3299.0,117195.0,1.68,3299.0,215584.0,1.79,3299.0,130310.0,1.83,3299.0,89850.0,1.83,3299.0,130322.0,NaN,3299.0,101382.0,NaN,3299.0,176466.0,1.80,3299.0,176092.0,NaN,3299.0,106988.0,NaN,3299.0,85458.0,NaN,3299.0,130301.0,1.83,3299.0,117195.0,1.68,3299.0,215584.0,1.79,3299.0,130310.0,1.83,3299.0,89850.0,1.83,3299.0,130322.0,NaN,3299.0,101382.0,NaN,3299.0,176466.0,1.80,3299.0,176092.0,NaN,3299.0,106988.0,NaN,3299.0,85458.0,NaN,3299.0,1
2,2928001,3375,17201,Freundschaftsspiele,NaN,1980-02-13,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2928001.0,135084.0,1.80,3375.0,117603.0,1.80,3375.0,159968.0,1.77,3375.0,135089.0,1.85,3375.0,135099.0,1.83,3375.0,141272.0,1.79,3375.0,135088.0,1.78,3375.0,117599.0,1.83,3375.0,135096.0,1.69,3375.0,141276.0,1.71,3375.0,135092.0,1.75,3375.0,135084.0,1.80,3375.0,117603.0,1.80,3375.0,159968.0,1.77,3375.0,135089.0,1.85,3375.0,135099.0,1.83,3375.0,141272.0,1.79,3375.0,135088.0,1.78,3375.0,117599.0,1.83,3375.0,135096.0,1.69,3375.0,141276.0,1.71,3375.0,135092.0,1.75,3375.0,0
3,2371640,3376,3447,Freundschaftsspiele,NaN,1980-02-16,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2371640.0,89229.0,NaN,3376.0,135787.0,1.80,3376.0,116689.0,NaN,3376.0,1288

In [24]:
#create classes for columns which arent numbers
competition_dummies = pd.get_dummies(prepared_matches_train['competition'], prefix='class', drop_first=True)
matchday_dummies = pd.get_dummies(prepared_matches_train['match_day'], prefix='class', drop_first=True)
confed_home_dummies = pd.get_dummies(prepared_matches_train['confederation_home'], prefix='class_home', drop_first=True)
confed_away_dummies = pd.get_dummies(prepared_matches_train['confederation_away'], prefix='class_away', drop_first=True)

prepared_matches_train = pd.concat([prepared_matches_train, competition_dummies, matchday_dummies, confed_home_dummies, confed_away_dummies],axis=1)

prepared_matches_train = prepared_matches_train.drop('competition', axis=1)
prepared_matches_train = prepared_matches_train.drop('match_day', axis=1)
prepared_matches_train = prepared_matches_train.drop('confederation_home', axis=1)
prepared_matches_train = prepared_matches_train.drop('confederation_away', axis=1)

prepared_matches_train.tail(10)

,id_int,id_int_home,id_int_away,date,goals_home,goals_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_date_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,rank_date_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,class_AFC Asian Cup 2007,class_AFC Asian Cup 2011,class_AFC Asian Cup 2015,class_AFC Challenge Cup 2014,class_AFC-OFC Challenge Cup,class_Afrika-Cup 1998,class_Afrika-Cup 2000,class_Afrika-Cup 2002,class_Afrika-Cup 2004,class_Afrika-Cup 2006,class_Afrika-Cup 2008,class_Afrika-Cup 2010,class_Afrika-Cup 2012,class_Afrika-Cup 2013,class_Afrika-Cup 2015,class_Afrika-Cup 2017,class_Afrika-Cup-Qualifikation,class_Artemio Franchi Trophy,class_Asian Cup Qualifikation,class_CONCACAF Cup,class_Caribbean Cup 2005,class_Caribbean Cup 2007,class_Caribbean Cup 2008,class_Caribbean Cup 2010,class_Caribbean Cup 2012,class_Caribbean Cup 2014,class_Caribbean Cup 2017,class_Caribbean Cup Qualifikation,class_Confederations Cup 1997,class_Confederations Cup 1999,class_Confederations Cup 2001,class_Confederations Cup 2003,class_Confederations Cup 2005,class_Confederations Cup 2009,class_Confederations Cup 2013,class_Confederations Cup 2017,class_Copa América 1983,class_Copa América 1987,class_Copa América 1989,class_Copa América 1991,class_Copa América 1993,class_Copa América 1995,class_Copa América 1997,class_Copa América 1999,class_Copa América 2001,class_Copa América 2004,class_Copa América 2007,class_Copa América 2011,class_Copa América 2015,class_Copa América Centenario 2016,class_Copa América Centenario Play-In,class_Copa Centroamericana 2011,class_Copa Centroamericana 2013,class_Copa Centroamericana 2014,class_Copa Centroamericana 2017,class_EM-Qualifikation,class_EM-Qualifikation Playoffs,class_Europameisterschaft 1980,class_Europameisterschaft 1984,class_Europameisterschaft 1988,class_Europameisterschaft 1992,class_Europameisterschaft 1996,class_Europameisterschaft 2000,class_Europameisterschaft 2004,class_Europameisterschaft 2008,class_Europameisterschaft 2012,class_Europameisterschaft 2016,class_Freundschaftsspiele,class_Gold Cup 1991,class_Gold Cup 1993,class_Gold Cup 1996,class_Gold Cup 1998,class_Gold Cup 2000,class_Gold Cup 2002,class_Gold Cup 2003,class_Gold Cup 2005,class_Gold Cup 2007,class_Gold Cup 2009,class_Gold Cup 2011,class_Gold Cup 2013,class_Gold Cup 2015,class_Gold Cup 2017,class_Gold Cup Playoff,class_Gold Cup Qualifikation,class_König-Fahd-Pokal 1992,class_König-Fahd-Pokal 1995,class

In [26]:
#change change date to days_until_wm (better for models), also convert datetime64 to float

prepared_matches_train["date"] = pd.to_datetime(prepared_matches_train["date"])
prepared_matches_train["rank_date_home"] = pd.to_datetime(prepared_matches_train["rank_date_home"])
prepared_matches_train["rank_date_away"] = pd.to_datetime(prepared_matches_train["rank_date_away"])

prepared_matches_train["game_days_before_wm"] = (np.datetime64('2018-06-14T00:00:00Z') - prepared_matches_train["date"]).dt.total_seconds() / (24 * 60 * 60)
prepared_matches_train["rank_home_days_before_wm"] = (np.datetime64('2018-06-14T00:00:00Z') - prepared_matches_train["rank_date_home"]).dt.total_seconds() / (24 * 60 * 60)
prepared_matches_train["rank_away_days_before_wm"] = (np.datetime64('2018-06-14T00:00:00Z') - prepared_matches_train["rank_date_away"]).dt.total_seconds() / (24 * 60 * 60)


prepared_matches_train = prepared_matches_train.drop('date', axis=1)
prepared_matches_train = prepared_matches_train.drop('rank_date_home', axis=1)
prepared_matches_train = prepared_matches_train.drop('rank_date_away', axis=1)


prepared_matches_train.tail(10)

/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  import sys
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  
/home/marco/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  if __name__ == '__main__':


,id_int,id_int_home,id_int_away,goals_home,goals_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,class_AFC Asian Cup 2007,class_AFC Asian Cup 2011,class_AFC Asian Cup 2015,class_AFC Challenge Cup 2014,class_AFC-OFC Challenge Cup,class_Afrika-Cup 1998,class_Afrika-Cup 2000,class_Afrika-Cup 2002,class_Afrika-Cup 2004,class_Afrika-Cup 2006,class_Afrika-Cup 2008,class_Afrika-Cup 2010,class_Afrika-Cup 2012,class_Afrika-Cup 2013,class_Afrika-Cup 2015,class_Afrika-Cup 2017,class_Afrika-Cup-Qualifikation,class_Artemio Franchi Trophy,class_Asian Cup Qualifikation,class_CONCACAF Cup,class_Caribbean Cup 2005,class_Caribbean Cup 2007,class_Caribbean Cup 2008,class_Caribbean Cup 2010,class_Caribbean Cup 2012,class_Caribbean Cup 2014,class_Caribbean Cup 2017,class_Caribbean Cup Qualifikation,class_Confederations Cup 1997,class_Confederations Cup 1999,class_Confederations Cup 2001,class_Confederations Cup 2003,class_Confederations Cup 2005,class_Confederations Cup 2009,class_Confederations Cup 2013,class_Confederations Cup 2017,class_Copa América 1983,class_Copa América 1987,class_Copa América 1989,class_Copa América 1991,class_Copa América 1993,class_Copa América 1995,class_Copa América 1997,class_Copa América 1999,class_Copa América 2001,class_Copa América 2004,class_Copa América 2007,class_Copa América 2011,class_Copa América 2015,class_Copa América Centenario 2016,class_Copa América Centenario Play-In,class_Copa Centroamericana 2011,class_Copa Centroamericana 2013,class_Copa Centroamericana 2014,class_Copa Centroamericana 2017,class_EM-Qualifikation,class_EM-Qualifikation Playoffs,class_Europameisterschaft 1980,class_Europameisterschaft 1984,class_Europameisterschaft 1988,class_Europameisterschaft 1992,class_Europameisterschaft 1996,class_Europameisterschaft 2000,class_Europameisterschaft 2004,class_Europameisterschaft 2008,class_Europameisterschaft 2012,class_Europameisterschaft 2016,class_Freundschaftsspiele,class_Gold Cup 1991,class_Gold Cup 1993,class_Gold Cup 1996,class_Gold Cup 1998,class_Gold Cup 2000,class_Gold Cup 2002,class_Gold Cup 2003,class_Gold Cup 2005,class_Gold Cup 2007,class_Gold Cup 2009,class_Gold Cup 2011,class_Gold Cup 2013,class_Gold Cup 2015,class_Gold Cup 2017,class_Gold Cup Playoff,class_Gold Cup Qualifikation,class_König-Fahd-Pokal 1992,class_König-Fahd-Pokal 1995,class_OFC Nations Cup 1996,class_OFC Nat

In [27]:
#convert the player objects to float
prepared_matches_train = prepared_matches_train.astype('float64')

pd.options.display.max_rows = 30#400
print(prepared_matches_train.dtypes)



id_int                              float64
id_int_home                         float64
id_int_away                         float64
goals_home                          float64
goals_away                          float64
rank_home                           float64
total_points_home                   float64
previous_points_home                float64
rank_change_home                    float64
cur_year_avg_home                   float64
cur_year_avg_weighted_home          float64
last_year_avg_home                  float64
last_year_avg_weighted_home         float64
two_year_ago_avg_home               float64
two_year_ago_weighted_home          float64
                                     ...   
class_Zwischenrunde - Hinspiele     float64
class_Zwischenrunde - Rückspiele    float64
class_home_CAF                      float64
class_home_CONCACAF                 float64
class_home_CONMEBOL                 float64
class_home_OFC                      float64
class_home_UEFA                 

In [28]:
#change all NaN-values to 0.0. Bad solution, there are other, better ways to go

prepared_matches_train = prepared_matches_train.fillna(0)
prepared_matches_train.tail(10)

,id_int,id_int_home,id_int_away,goals_home,goals_away,rank_home,total_points_home,previous_points_home,rank_change_home,cur_year_avg_home,cur_year_avg_weighted_home,last_year_avg_home,last_year_avg_weighted_home,two_year_ago_avg_home,two_year_ago_weighted_home,three_year_ago_avg_home,three_year_ago_weighted_home,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,three_year_ago_avg_away,three_year_ago_weighted_away,id_int_match,hp_1_id_int_player,hp_1_height,hp_1_country_id_int,hp_2_id_int_player,hp_2_height,hp_2_country_id_int,hp_3_id_int_player,hp_3_height,hp_3_country_id_int,hp_4_id_int_player,hp_4_height,hp_4_country_id_int,hp_5_id_int_player,hp_5_height,hp_5_country_id_int,hp_6_id_int_player,hp_6_height,hp_6_country_id_int,hp_7_id_int_player,hp_7_height,hp_7_country_id_int,hp_8_id_int_player,hp_8_height,hp_8_country_id_int,hp_9_id_int_player,hp_9_height,hp_9_country_id_int,hp_10_id_int_player,hp_10_height,hp_10_country_id_int,hp_11_id_int_player,hp_11_height,hp_11_country_id_int,ap_1_id_int_player,ap_1_height,ap_1_country_id_int,ap_2_id_int_player,ap_2_height,ap_2_country_id_int,ap_3_id_int_player,ap_3_height,ap_3_country_id_int,ap_4_id_int_player,ap_4_height,ap_4_country_id_int,ap_5_id_int_player,ap_5_height,ap_5_country_id_int,ap_6_id_int_player,ap_6_height,ap_6_country_id_int,ap_7_id_int_player,ap_7_height,ap_7_country_id_int,ap_8_id_int_player,ap_8_height,ap_8_country_id_int,ap_9_id_int_player,ap_9_height,ap_9_country_id_int,ap_10_id_int_player,ap_10_height,ap_10_country_id_int,ap_11_id_int_player,ap_11_height,ap_11_country_id_int,home_is_winner,class_AFC Asian Cup 2007,class_AFC Asian Cup 2011,class_AFC Asian Cup 2015,class_AFC Challenge Cup 2014,class_AFC-OFC Challenge Cup,class_Afrika-Cup 1998,class_Afrika-Cup 2000,class_Afrika-Cup 2002,class_Afrika-Cup 2004,class_Afrika-Cup 2006,class_Afrika-Cup 2008,class_Afrika-Cup 2010,class_Afrika-Cup 2012,class_Afrika-Cup 2013,class_Afrika-Cup 2015,class_Afrika-Cup 2017,class_Afrika-Cup-Qualifikation,class_Artemio Franchi Trophy,class_Asian Cup Qualifikation,class_CONCACAF Cup,class_Caribbean Cup 2005,class_Caribbean Cup 2007,class_Caribbean Cup 2008,class_Caribbean Cup 2010,class_Caribbean Cup 2012,class_Caribbean Cup 2014,class_Caribbean Cup 2017,class_Caribbean Cup Qualifikation,class_Confederations Cup 1997,class_Confederations Cup 1999,class_Confederations Cup 2001,class_Confederations Cup 2003,class_Confederations Cup 2005,class_Confederations Cup 2009,class_Confederations Cup 2013,class_Confederations Cup 2017,class_Copa América 1983,class_Copa América 1987,class_Copa América 1989,class_Copa América 1991,class_Copa América 1993,class_Copa América 1995,class_Copa América 1997,class_Copa América 1999,class_Copa América 2001,class_Copa América 2004,class_Copa América 2007,class_Copa América 2011,class_Copa América 2015,class_Copa América Centenario 2016,class_Copa América Centenario Play-In,class_Copa Centroamericana 2011,class_Copa Centroamericana 2013,class_Copa Centroamericana 2014,class_Copa Centroamericana 2017,class_EM-Qualifikation,class_EM-Qualifikation Playoffs,class_Europameisterschaft 1980,class_Europameisterschaft 1984,class_Europameisterschaft 1988,class_Europameisterschaft 1992,class_Europameisterschaft 1996,class_Europameisterschaft 2000,class_Europameisterschaft 2004,class_Europameisterschaft 2008,class_Europameisterschaft 2012,class_Europameisterschaft 2016,class_Freundschaftsspiele,class_Gold Cup 1991,class_Gold Cup 1993,class_Gold Cup 1996,class_Gold Cup 1998,class_Gold Cup 2000,class_Gold Cup 2002,class_Gold Cup 2003,class_Gold Cup 2005,class_Gold Cup 2007,class_Gold Cup 2009,class_Gold Cup 2011,class_Gold Cup 2013,class_Gold Cup 2015,class_Gold Cup 2017,class_Gold Cup Playoff,class_Gold Cup Qualifikation,class_König-Fahd-Pokal 1992,class_König-Fahd-Pokal 1995,class_OFC Nations Cup 1996,class_OFC Nat

In [29]:
#save to csv
prepared_matches_train.to_csv("prepared_matches_train.csv", encoding='utf-8')